In [0]:

import pickle, csv
import numpy as np
import pandas as pd

import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

**setting the parameters**

In [0]:
num_epochs = 50;
batch_size = 50;
learning_rate = 0.001;

In [0]:
class pkl_dataset(Dataset):
    '''taking the data'''
    def __init__(self, image_path, label_path):
        '''
        Parameters
        ----------
        image_path: Path to pickle file with training images
        
        label_path: Patht to pickle file with corresponding labels
        '''
        # Read pickle file and processing
        data = pickle.load(open(image_path,"rb"))
        self.X = np.array(data).reshape(-1, 1, 28, 28)

        # Read pickle file and processing
        label = pd.DataFrame(pickle.load(open(label_path,"rb")))
        npLabel = np.asarray(label)

        # labels as follows:
        # 0 -> 0
        # 2 -> 1
        # 3 -> 2
        # 6 -> 3
        np.place(npLabel, npLabel == 0, 0)
        np.place(npLabel, npLabel == 2, 1)
        np.place(npLabel, npLabel == 3, 2)
        np.place(npLabel, npLabel == 6, 3)
        self.Y = np.array(label.iloc[:, 0]);
        
        del data;
        
    def __len__(self):
        return len(self.X);
    
    def __getitem__(self, idx):
        item = self.X[idx];
        label = self.Y[idx];        
        return (item, label);

class pkl_test_dataset(Dataset):
    '''Processing given pickle dataset without labels (ie. Test data)'''
    def __init__(self, img_path):
        '''
        Parameters
        ----------
        image_path: Path to pickle file with testing images
        '''
        # Read pickle file and preprocess
        data = pickle.load(open(img_path,"rb"))
        self.X = np.array(data).reshape(-1, 1, 28, 28)
        
        del data;        

    def __len__(self):
        return len(self.X);
    
    def __getitem__(self, idx):
        item = self.X[idx];
                
        return (item);

# Train and test Dataset Object


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os

image_path = '/content/drive/My Drive/Vision_task_dataset_public/train_image.pkl'
image_path1='/content/drive/My Drive/Vision_task_dataset_public/train_label.pkl'
test_path= '/content/drive/My Drive/Vision_task_dataset_public/test_image.pkl'
train = pkl_dataset(image_path,image_path1)
test = pkl_test_dataset(test_path)



In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    '''Deep Learning Network'''
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc1 = nn.Linear(7*7*32, 100)
        self.fc2 = nn.Linear(100, 4)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc2(self.fc1(out))
        return out

cnn = CNN()

In [0]:
import torch.optim as optim
import torch.autograd.variable as Variable

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate);

In [12]:
# Training dataset into test and train
train_size = int(0.8 * 8000)
test_size = 8000 - train_size


train_loader = torch.utils.data.DataLoader(dataset=train,
                                           batch_size=batch_size,
                                           shuffle=True);

losses = [];
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.float())
        labels = Variable(labels)
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item());
        
        if (i+1) % batch_size == 0:
            print ('Epoch : %d/%d, Iter : %d/%d,  Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, 8000//batch_size, loss.item()))

/usr/local/lib/python3.6/dist-packages/torch/autograd/__init__.py:167: UserWarning: torch.autograd.variable(...) is deprecated, use torch.tensor(...) instead
  warnings.warn("torch.autograd.variable(...) is deprecated, use torch.tensor(...) instead")
/usr/local/lib/python3.6/dist-packages/torch/autograd/__init__.py:168: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(*args, **kwargs)


Epoch : 1/50, Iter : 50/160,  Loss: 0.5729
Epoch : 1/50, Iter : 100/160,  Loss: 0.4608
Epoch : 1/50, Iter : 150/160,  Loss: 0.6136
Epoch : 2/50, Iter : 50/160,  Loss: 0.1976
Epoch : 2/50, Iter : 100/160,  Loss: 0.2952
Epoch : 2/50, Iter : 150/160,  Loss: 0.3012
Epoch : 3/50, Iter : 50/160,  Loss: 0.4030
Epoch : 3/50, Iter : 100/160,  Loss: 0.4692
Epoch : 3/50, Iter : 150/160,  Loss: 0.4027
Epoch : 4/50, Iter : 50/160,  Loss: 0.3108
Epoch : 4/50, Iter : 100/160,  Loss: 0.3221
Epoch : 4/50, Iter : 150/160,  Loss: 0.4565
Epoch : 5/50, Iter : 50/160,  Loss: 0.1416
Epoch : 5/50, Iter : 100/160,  Loss: 0.3460
Epoch : 5/50, Iter : 150/160,  Loss: 0.1890
Epoch : 6/50, Iter : 50/160,  Loss: 0.1347
Epoch : 6/50, Iter : 100/160,  Loss: 0.2155
Epoch : 6/50, Iter : 150/160,  Loss: 0.1915
Epoch : 7/50, Iter : 50/160,  Loss: 0.2232
Epoch : 7/50, Iter : 100/160,  Loss: 0.3310
Epoch : 7/50, Iter : 150/160,  Loss: 0.2956
Epoch : 8/50, Iter : 50/160,  Loss: 0.1577
Epoch : 8/50, Iter : 100/160,  Loss: 0.1

LOAD AND TEST THE DATASET

In [13]:
# Load test data
test_loader = torch.utils.data.DataLoader(dataset=test,
                                           batch_size=batch_size,
                                           shuffle=False);
# Run the model on the Test data
LA = list()
with torch.no_grad():
    for i, (images) in enumerate(test_loader):

        images = Variable(images.float())
        
        val, labels = torch.max(cnn(images), 1)
        LA.append(list(labels))

# Convert the labels back again
LA = np.asarray(LA).flatten()
np.place(LA, LA == 0, 0)
np.place(LA, LA == 1, 2)
np.place(LA, LA == 2, 3)
np.place(LA, LA == 3, 6)

/usr/local/lib/python3.6/dist-packages/torch/autograd/__init__.py:167: UserWarning: torch.autograd.variable(...) is deprecated, use torch.tensor(...) instead
  warnings.warn("torch.autograd.variable(...) is deprecated, use torch.tensor(...) instead")
/usr/local/lib/python3.6/dist-packages/torch/autograd/__init__.py:168: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(*args, **kwargs)


SAVINNG THE OUTPUT INTO TEST FILE

In [0]:

with open('Shashank_verma.csv', mode='w') as label_file:
    label_file = csv.writer(label_file, delimiter=',')
    label_file.writerow(['image_index', 'class'])
    for i in range(len(LA)):
        label_file.writerow([i, LA[i]])